In [3]:
import time
import json
import random
import datetime
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

# 🔍 체코 주요 뉴스 사이트 목록
czech_news_sites = [
    "site:idnes.cz", "site:novinky.cz", "site:aktualne.cz", "site:lidovky.cz",
    "site:seznamzpravy.cz", "site:ct24.ceskatelevize.cz", "site:hn.cz", "site:blesk.cz",
    "site:denik.cz", "site:respekt.cz", "site:reflex.cz", "site:euro.cz", "site:e15.cz",
    "site:info.cz", "site:tyden.cz", "site:echo24.cz", "site:forum24.cz"
]

# 🔍 검색어 설정 (체코 뉴스 검색)
search_query = "Jaderná elektrárna Dukovany"

# ✅ 3개월 단위로 검색할 날짜 범위 설정 (2022.01.01 ~ 2024.12.31)
date_ranges = [
    ("2022-01-01", "2022-03-31"), ("2022-04-01", "2022-06-30"),
    ("2022-07-01", "2022-09-30"), ("2022-10-01", "2022-12-31"),
    ("2023-01-01", "2023-03-31"), ("2023-04-01", "2023-06-30"),
    ("2023-07-01", "2023-09-30"), ("2023-10-01", "2023-12-31"),
    ("2024-01-01", "2024-03-31"), ("2024-04-01", "2024-06-30"),
    ("2024-07-01", "2024-09-30"), ("2024-10-01", "2024-12-31")
]

# ✅ 구글 뉴스 검색 URL 템플릿 (3개월 단위 검색)
GOOGLE_NEWS_URL_TEMPLATE = (
    "https://www.google.com/search?q={query}+{sites}&hl=cs&gl=CZ&tbm=nws&tbs=cdr:1,cd_min:{start_date},cd_max:{end_date},sbd:1&start={start}"
)

# ✅ User-Agent 설정 (랜덤 선택)
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36"
]

def fetch_google_news(start_date, end_date):
    """구글 뉴스에서 특정 기간 내 체코 뉴스 사이트 기사 링크 크롤링"""
    news_list = []
    search_sites = " OR ".join(czech_news_sites)

    for page in range(0, 1000, 10):  # 10개씩 로드, 최대 1000개 (100페이지)
        headers = {"User-Agent": random.choice(USER_AGENTS)}  # User-Agent 랜덤 변경
        search_url = GOOGLE_NEWS_URL_TEMPLATE.format(
            query=search_query, sites=search_sites,
            start_date=start_date, end_date=end_date, start=page
        )
        
        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.select("div.dbsr")  # 구글 뉴스 기사 컨테이너

        for article in articles:
            try:
                title = article.select_one("div.JheGif.nDgy9d").text.strip()
                link = article.a["href"]
                date = article.select_one("span.WG9SHc span").text.strip()

                news_list.append({
                    "title": title,
                    "link": link,
                    "date": date
                })
            except Exception as e:
                print(f"⚠ 오류 발생 (기사 링크 크롤링 실패): {e}")

        print(f"✅ {len(news_list)}개의 기사 링크 수집 완료 (페이지 {page//10+1})")

        # 페이지에 더 이상 기사가 없으면 중단
        if len(articles) == 0:
            break

        # 요청 간격 랜덤 설정 (3~6초)
        time.sleep(random.uniform(3, 6))

    return news_list

def fetch_news_content(news_list):
    """기사 본문 크롤링"""
    news_data = []

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        try:
            headers = {"User-Agent": random.choice(USER_AGENTS)}
            response = requests.get(news["link"], headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # ✅ 본문 크롤링 (다양한 선택자 추가)
            content_selectors = ["article", "div#newsct_article", "div#articleBodyContents", "div.article_view"]
            content = ""
            for selector in content_selectors:
                content_element = soup.select(selector)
                if content_element:
                    content = " ".join([p.text.strip() for p in content_element])
                    break

            # ✅ 데이터 저장
            news["content"] = content.strip() if content else "N/A"
            news_data.append(news)

            print(f"✅ 크롤링 완료: {news['title'][:50]}...")

            # 요청 간격 랜덤 설정 (2~5초)
            time.sleep(random.uniform(2, 5))

        except Exception as e:
            print(f"⚠ 오류 발생 (뉴스 크롤링 실패): {news['link']}, 오류: {e}")

    return news_data

# ✅ 구글 뉴스 크롤링 실행 (3개월 단위로 반복)
all_news = []
for start_date, end_date in date_ranges:
    print(f"📅 {start_date} ~ {end_date} 기간 크롤링 시작")
    news_list = fetch_google_news(start_date, end_date)
    news_data = fetch_news_content(news_list)
    all_news.extend(news_data)

# ✅ JSON 파일로 저장
output_file = "google_news_czech.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_news, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(all_news)}개의 뉴스 기사를 저장했습니다. (파일명: {output_file})")


📅 2022-01-01 ~ 2022-03-31 기간 크롤링 시작
✅ 0개의 기사 링크 수집 완료 (페이지 1)


📰 뉴스 본문 크롤링 진행: 0it [00:00, ?it/s]

📅 2022-04-01 ~ 2022-06-30 기간 크롤링 시작


✅ 0개의 기사 링크 수집 완료 (페이지 1)


📰 뉴스 본문 크롤링 진행: 0it [00:00, ?it/s]

...

📰 뉴스 본문 크롤링 진행: 0it [00:00, ?it/s]

📅 2024-10-01 ~ 2024-12-31 기간 크롤링 시작


✅ 0개의 기사 링크 수집 완료 (페이지 1)


📰 뉴스 본문 크롤링 진행: 0it [00:00, ?it/s]

✅ 총 0개의 뉴스 기사를 저장했습니다. (파일명: google_news_czech.json)
